# Generates the JSON manifests that are used to run the SAILOR pipeline
- Use sailor/1.1.0 so that we can capture C/T edits. The current released version only does A/G

In [1]:
import yaml
import os
import pandas as pd
import glob
from qtools import Submitter
from tqdm import tnrange, tqdm_notebook
import time

In [2]:
input_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/05_scRNA_RBFOX2_APO_transient/bam_files'
output_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/05_scRNA_RBFOX2_APO_transient/tmp/sailor_outputs_individual_barcodes'

ref_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/05_scRNA_RBFOX2_APO_transient/inputs'

In [3]:
### I'm re-running this notebook a few times on a few different groups of bam files
bams = glob.glob(os.path.join(input_dir, '*-1.bam'))
print(len(bams))
bams[:3]

19156


['/home/bay001/projects/kris_apobec_20200121/permanent_data2/05_scRNA_RBFOX2_APO_transient/bam_files/RBFOX2_transient_possorted_genome_bam_MD-CTTTCAACACTTTAGG-1.bam',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/05_scRNA_RBFOX2_APO_transient/bam_files/RBFOX2_transient_possorted_genome_bam_MD-CCAATGAGTTCGGGTC-1.bam',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/05_scRNA_RBFOX2_APO_transient/bam_files/RBFOX2_transient_possorted_genome_bam_MD-GTGATGTCAATGTTGC-1.bam']

In [4]:
for nt in ['A', 'C', 'G', 'T']:
    for nt2 in ['A', 'C', 'G', 'T']:
        if not os.path.exists(output_dir + '_{}{}'.format(nt, nt2)):
            split_output = os.path.join(output_dir + '_{}{}'.format(nt, nt2))
            ! mkdir $split_output

# This block generates SAILOR json manifest runner files.

In [5]:
def return_genome_file(fn):
    if os.path.basename(fn)[:6] == 'RBFOX2':
        return os.path.join(ref_dir, 'refdata-cellranger-hg19_APOBEC_RBFOX2_transient/fasta/genome.fa')
    elif os.path.basename(fn)[:6] == 'APOBEC':
        return os.path.join(ref_dir, 'refdata-cellranger-hg19_APOBEC_transient/fasta/genome.fa')
    else:
        print("{}".format(os.path.basename(fn)))
        return 1
    
skip_rmdup = "true"
progress = tnrange(len(bams))
for bam in bams:
    genome_fa = return_genome_file(bam)
    if genome_fa == 1:
        break
    prefix = os.path.basename(bam).split('_MD-')[1][:2]
    split_output_dir = output_dir + "_{}".format(prefix)
    
    sample = {"input_bam": {
        'class':'File',
        'path':bam
    }}
    fa = {"reference": {
        'class':'File',
        'path':genome_fa
    }}
    known_snp = {"known_snp": {
        'class':'File',
        'path':os.path.join(ref_dir, 'hg19.commonSNPs147.bed3')
    }}
    with open(
        os.path.join(
            split_output_dir, '{}.json'.format(
                os.path.basename(bam).split('.')[0]
            )
        ), 'w'
    ) as o:
        o.write("#!/usr/bin/env SAILOR\n")
        yaml.dump(sample, o, default_flow_style=False)
        yaml.dump(known_snp, o, default_flow_style=False)
        yaml.dump(fa, o, default_flow_style=False)
        o.write("ct: true\n")
        o.write("min_variant_coverage: 5\n")
        o.write("alpha: {}\n".format(0))
        o.write("beta: {}\n".format(0))
        o.write("edit_fraction: {}\n".format(0.01))
        o.write("skip_duplicate_removal: {}\n".format(skip_rmdup))
        o.write("reverse_stranded_library: false\n")
    progress.update(1)

# Grab all JSON files created from the above cell.

In [6]:
jsons = []
for nt in ['A', 'C', 'G', 'T']:
    for nt2 in ['A', 'C', 'G', 'T']:
        split_output = os.path.join(output_dir + '_{}{}'.format(nt, nt2))
        jsons += glob.glob(os.path.join(split_output, '*.json'))
print(len(jsons))

19156


In [7]:
jsons[:3]

['/home/bay001/projects/kris_apobec_20200121/permanent_data2/05_scRNA_RBFOX2_APO_transient/tmp/sailor_outputs_individual_barcodes_AA/RBFOX2_transient_possorted_genome_bam_MD-AATTCCTTCATTCATC-1.json',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/05_scRNA_RBFOX2_APO_transient/tmp/sailor_outputs_individual_barcodes_AA/RBFOX2_transient_possorted_genome_bam_MD-AACCACAAGGCTTAAA-1.json',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data2/05_scRNA_RBFOX2_APO_transient/tmp/sailor_outputs_individual_barcodes_AA/RBFOX2_transient_possorted_genome_bam_MD-AATTTCCCACTGTCGG-1.json']

In [8]:
cmds = []

def chunker(seq, size):
    """
    Chunks a long list into groups of (size).
    """
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

groupsize = 50 # 50 sailor runs per job.
progress = tnrange(len(jsons))
for json_group in chunker(jsons, groupsize):
    cmd = 'module load sailor/1.1.0;'
    for json in json_group:
        prefix = os.path.basename(json).split('_MD-')[1][:2]
        split_output_dir = output_dir + "_{}".format(prefix)
        ! chmod +x $json
        cmd += 'cd {};'.format(split_output_dir)
        cmd += './{};'.format(os.path.basename(json))
        progress.update(1)
    cmds.append(cmd)
cmds[:2]

['module load sailor/1.1.0;cd /home/bay001/projects/kris_apobec_20200121/permanent_data2/05_scRNA_RBFOX2_APO_transient/tmp/sailor_outputs_individual_barcodes_AA;./RBFOX2_transient_possorted_genome_bam_MD-AATTCCTTCATTCATC-1.json;cd /home/bay001/projects/kris_apobec_20200121/permanent_data2/05_scRNA_RBFOX2_APO_transient/tmp/sailor_outputs_individual_barcodes_AA;./RBFOX2_transient_possorted_genome_bam_MD-AACCACAAGGCTTAAA-1.json;cd /home/bay001/projects/kris_apobec_20200121/permanent_data2/05_scRNA_RBFOX2_APO_transient/tmp/sailor_outputs_individual_barcodes_AA;./RBFOX2_transient_possorted_genome_bam_MD-AATTTCCCACTGTCGG-1.json;cd /home/bay001/projects/kris_apobec_20200121/permanent_data2/05_scRNA_RBFOX2_APO_transient/tmp/sailor_outputs_individual_barcodes_AA;./APOBEC_transient_possorted_genome_bam_MD-AAGACTCTCGTCGACG-1.json;cd /home/bay001/projects/kris_apobec_20200121/permanent_data2/05_scRNA_RBFOX2_APO_transient/tmp/sailor_outputs_individual_barcodes_AA;./APOBEC_transient_possorted_genome

In [9]:
print("total commands: {}".format(len(cmds)))

total commands: 384


In [10]:
Submitter(
    commands=cmds,
    job_name='scRNA-sailor-individual',
    sh='scRNA-sailor-individual.sh',
    array=True,
    nodes=1,
    ppn=1,
    walltime='72:00:00',
    submit=True,
)

Writing 384 tasks as an array-job.
Wrote commands to scRNA-sailor-individual.sh.
Submitted script to queue home.
 Job ID: 21376044
